# Python USB library for the Consumer Physics SCIO

__*IMPORTANT NOTE*__: This file contains the entire library (for testing and development purposes), but it is advisable to instead import the library. Refer to the other Jupyter notebook for that use.

### Dependencies
- pySerial

In [10]:
import json
import struct
import logging
import serial
import serial.tools.list_ports as pyserial
#from serial.tools import list_ports_common # Not sure I need this?

# Logging/output format setup
log = logging.getLogger('root')
log.setLevel(logging.DEBUG)
logging.basicConfig(format='[%(asctime)s] %(levelname)8s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [13]:
# Search for device
def search_scio_usb(silent=False):
    if not silent: print('Searching for devices:')
    scio_ID = '0451:16AA'
    device_list = []
    # Find the scio port. Returns an empty list if not found
    scio_port = list(pyserial.grep(scio_ID))
    if(len(scio_port) == 0):
        print('  - Device not found. Try a long press (device should respond with blue blinking)')
        return(device_list)
    device_list.append({'address': scio_port[0].device, 'id': scio_ID})

    return(device_list)

class scio_usb:
    def __init__(self, address):
        self.address = address
        self.client = None
        self.disconnected = True
        self.cmd_base_protocol = -70
        self.cmd_temp = 4
        self.cmd_scan = 2
        
    async def connect(self):
        try:
            self.client = serial.Serial(self.address)
            self.disconnected = False
        except Exception as e:
            logging.error(f'Failed to connect to {self.address}: {e}')
            raise
        pass
    
    async def disconnect(self):
        if not self.disconnected:
            try:
                self.client.close()
            except Exception as e:
                logging.error(f'Failed to disconnect from {self.address}: {e}')
                raise
            finally:
                self.disconnected = True
        pass
        
    def create_command(self, cmd):
        if(cmd == SET_LED):
            # Setting the LED colour is special (longer command)
            byte_cmd = struct.pack('<bbbbbbbbbbbbbbb',1,self.cmd_base_protocol,cmd,9,0,0,0,0,0,0,0,0,0,0)
        else:
            byte_cmd = struct.pack('<bbbbb',1,self.cmd_base_protocol,cmd,0,0)
        return(byte_cmd)
    
    async def send_command(self, byte_cmd):
        try:
            self.client.write(byte_cmd)
        except Exception as e:
            logging.error(f'Failed to send command to {self.address}: {e}')
            raise
        pass
    
    async def read_response(self):
        try:
            pass# TODO
        except Exception as e:
            logging.error(f'Failed to read command from {self.address}: {e}')
            raise
        pass
    
    def parse_response(self, raw_response):
        # TODO
        return(raw_response)
    
    async def read_test(self):
        service_uuid = self.uuid_test
        current_service = self.base_service_uuid.replace('xxxx', service_uuid)
        
        try:
            test = await self.client.read_gatt_char(current_service)
            return test
        except Exception as e:
            logging.error(f'Failed to model nb from {self.address}: {e}')
            raise
        pass
    
    async def run(self, logfile, silent=True):
        if not silent: print('  - Connecting to device')
        await self.connect()
        
        # TEST
        #try:
        #    test = await self.read_test()
        #except:
        #    pass
        #print(len(test))
        #print(struct.unpack('<II', test))
        
        # Tell it to read temperature
        #await self.cmd_temperature()
        # Read the reply
        #test = await self.read_cmd_reply()
        #print(test)
        
        
        if not silent: print('  - Disconnecting')
        await self.disconnect()
        
        return

In [15]:
logfile = './logfile.csv'

# Search for devices
device_list = search_scio_usb()
if(len(device_list) == 0):
    print('  - Device not found, make sure it is broadcasting through a long press')
else:
    # Note: No support for multiple Apogee devices. This takes the first one found!
    device_address = device_list[0]['address']
    print('  - Found device at', device_address)
    # Now set up device
    scio = scio_usb(device_address) # Create an instance for this device address
    await scio.run(logfile, silent=False)
print('Done...')

Searching for devices:
  - Found device at  COM5
  - Connecting to device
  - Disconnecting
Done...


In [ ]:
device_list = await search_scio_ble(3)
print(device_list)